### Shapash Model Overview
https://shapash.readthedocs.io/en/latest/

##### With this tutorial you:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]

In [5]:
X.head()

,day,sex,size,smoker,time,total_bill
0,Sun,Female,2,No,Dinner,16.99
1,Sun,Male,3,No,Dinner,10.34
2,Sun,Male,3,No,Dinner,21.01
3,Sun,Male,2,No,Dinner,23.68
4,Sun,Female,4,No,Dinner,24.59


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

C:\Users\ASHISH KUMAR\AppData\Local\Temp\ipykernel_4312\4105089070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['day']=X['day'].cat.codes
C:\Users\ASHISH KUMAR\AppData\Local\Temp\ipykernel_4312\4105089070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sex']=X['sex'].cat.codes
C:\Users\ASHISH KUMAR\AppData\Local\Temp\ipykernel_4312\4105089070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [8]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [9]:
### Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [11]:
from shapash.explainer.smart_explainer import SmartExplainer

In [26]:
xpl = SmartExplainer(model=regressor)

In [13]:
xpl.compile(x= X_test)

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x000002764929EB10>


In [14]:
xpl

#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [15]:
app = xpl.run_app(title_story='Tips Dataset')

C:\Users\ASHISH KUMAR\anaconda3\Lib\site-packages\shapash\webapp\smart_app.py:124: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\ASHISH KUMAR\anaconda3\Lib\site-packages\shapash\webapp\utils\utils.py:40: FutureWarning:

Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead

INFO:root:Your Shapash application run on http://DESKTOP-EK7R1VO:8050/
INFO:root:Use the method .kill() to down your app.


In [16]:
predictor = xpl.to_smartpredictor()

In [17]:
predictor.save('./predictor.pkl')

In [18]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [19]:
predictor_load.add_input(x=X, ypred=y)

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x0000027649AD7B90>


In [20]:
detailed_contributions = predictor_load.detail_contributions()


INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x0000027649AD7B90>


In [21]:
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.077967,0.143251,-0.005563,-0.023885,-0.005439,0.182180
1,1.66,0.094424,-0.051238,0.001211,-0.068126,-0.018199,-1.253710
2,3.50,0.081522,-0.021027,-0.010045,-0.000808,0.009520,0.439100
3,3.31,0.165301,-0.018933,-0.023320,0.020757,0.004276,0.116381
4,3.61,0.057116,0.059607,-0.043182,0.005242,-0.006417,-0.339905


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [22]:

predictor_load.modify_mask(max_contrib=3)

In [23]:
explanation = predictor_load.summarize()

C:\Users\ASHISH KUMAR\anaconda3\Lib\site-packages\shapash\manipulation\summarize.py:93: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [24]:
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.18218,sex,1.0,0.143251,day,3.0,0.077967
1,1.66,total_bill,10.34,-1.25371,day,3.0,0.094424,smoker,1.0,-0.068126
2,3.50,total_bill,21.01,0.4391,day,3.0,0.081522,sex,0.0,-0.021027
3,3.31,day,3.0,0.165301,total_bill,23.68,0.116381,size,2.0,-0.02332
4,3.61,total_bill,24.59,-0.339905,sex,1.0,0.059607,day,3.0,0.057116
